## STRATEGY 

Parameters:
	Stretch:	0.5
	LongTermMALen:	200
	ShortTermMALen:	20
		
Data:
	ATRValue:	ATR(3)
	ADXValue:	ADX(5)
	
	ShortTermMA:	MA(Close, ShortTermMALen)
	LongTermMA:	MA(Close, LongTermMALen)

	ClosingRange:	(Close - Low) / (High - Low)
	IsUptrend: 	Close > LongTermMA
	IsVolatile:	ADXValue > 30			  

	Long_limit:	    Low - (ATRValue * Stretch) // limit order price
	Long_trigger:	ClosingRange < 0.3 // close in lower 30% of bar
	Long_setup:	    LongTrigger and IsUptrend and IsLiquid and IsVolatile
	
	StopLossExit:	Close < ShortTermMA



	Exit_rule:	Close > FillPrice or StopLossExit
	Exit_price:	NextOpen

1. go through and simplify or explain with comments
1b. try to use pairname to access data for better readability
2. use try to create object (not to long)
3. start process from beginning to see how repeadble it is 

In [118]:
import vectorbt as vbt
import pandas as pd 
import numpy as np
from datetime import datetime
from kucoin_candle_spot import SpotDataFetcher
from datetime import datetime, timezone
import pandas_ta as ta
from numba import njit
import plotly.graph_objs as go


## data collection

In [119]:

symbols = ["ETH-USDT", "BTC-USDT"]
timeframe = "1hour"
start_time = "2025-01-08 10:00:00"
end_time = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")


# Fetch data for each symbol and store in a dictionary
dataframes = {}
for symbol in symbols:
    fetcher = SpotDataFetcher(symbol, timeframe, start_time, end_time)
    df = fetcher.fetch_candles_as_df()
    df['symbol'] = symbol
    dataframes[symbol] = df

# Print the DataFrame for each symbol
for symbol, df in dataframes.items():
    print(f"\nDataFrame for {symbol}:")
    print(df)
# fetcher = SpotDataFetcher(symbol, timeframe, start_time, end_time)




INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 1, Candles: 554
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 1, Candles: 554



DataFrame for ETH-USDT:
                              open    close     high      low         volume  \
timestamp                                                                      
2025-01-08 10:00:00+00:00  3347.14  3362.51  3373.15  3346.65   1480.4696085   
2025-01-08 11:00:00+00:00  3362.50  3349.35  3367.88  3323.31   2987.2982615   
2025-01-08 12:00:00+00:00  3349.35  3339.64  3360.95  3338.81   2466.9161027   
2025-01-08 13:00:00+00:00  3339.64  3358.53  3368.55  3339.64  3854.46036772   
2025-01-08 14:00:00+00:00  3358.53  3369.70  3381.46  3347.83   3661.1849986   
...                            ...      ...      ...      ...            ...   
2025-01-31 07:00:00+00:00  3262.74  3241.78  3276.76  3239.45   1215.8891971   
2025-01-31 08:00:00+00:00  3241.79  3236.97  3248.54  3234.10    997.2620623   
2025-01-31 09:00:00+00:00  3236.98  3266.47  3270.71  3234.60    1268.281276   
2025-01-31 10:00:00+00:00  3266.04  3269.97  3271.92  3256.71    898.0735499   
2025-01-31 11:0

In [120]:
def custom_trading_strategy(close, high, low, stretch=0.5, 
                             long_term_ma_len=200, short_term_ma_len=20, adx_len=5, atr_len=3, symbol=None):
    # Calculate indicators
    atr = vbt.IndicatorFactory.from_talib('ATR').run(high, low, close, timeperiod=atr_len).real.to_numpy()
    adx = vbt.IndicatorFactory.from_talib('ADX').run(high, low, close, timeperiod=adx_len).real.to_numpy()
    
    # Moving Averages
    long_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=long_term_ma_len).real.to_numpy()
    short_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=short_term_ma_len).real.to_numpy()

    # Closing Range
    closing_range = (close - low) / (high - low)
    
    # Conditions
    is_uptrend = close > long_term_ma
    is_volatile = adx > 30
    
    # Long Limit
    long_limit = low - (atr * stretch)
    
    # Long Trigger
    long_trigger = closing_range < 0.3
    
    # Long Setup
    long_setup = long_trigger & is_uptrend & is_volatile
    
    # Return limit order price when conditions for long setup are met
    limit_order_price = np.where(long_setup, long_limit, np.nan)
    
    # Ensure the order of outputs matches output_names
    return limit_order_price, long_term_ma,short_term_ma, atr, adx

# Create the indicator factory
custom_strategy_indicator = vbt.IndicatorFactory(
    class_name='CustomTradingStrategy',
    short_name='custom_strategy',
    input_names=['Close', 'High', 'Low'],
    param_names=['stretch', 'long_term_ma_len','short_term_ma_len', 'adx_len', 'atr_len','symbol'],
    output_names=['limit_order_price' ,'long_term_ma','short_term_ma', 'atr', 'adx']  # Add outputs for indicators

).from_apply_func(custom_trading_strategy)




In [121]:
# Entry parameter 
stretch = 0.5
long_term_ma_len = [100,200]
adx_len = [5,6]
atr_len = 3

# exit parameter
short_term_ma_len = [20,30,40]


indicator_obj = {}
for symbol in symbols:
    print(symbol)
    df = dataframes[symbol]

    # Usage example (commented out)
    indicator = custom_strategy_indicator.run(
        df['close'], df['high'], df['low'],
        stretch=stretch,
        long_term_ma_len=long_term_ma_len,
        short_term_ma_len=short_term_ma_len,
        adx_len=adx_len,
        atr_len=atr_len,
        symbol=symbol,
        param_product=True

    )

    indicator_obj[symbol] = indicator

indicator_obj




ETH-USDT
BTC-USDT


In [145]:
def copy_rename_close_series(df_original, df_indicator_signals):

    def repeat_series_horizontally(series, target_shape):
        return np.tile(series.values.reshape(-1, 1), target_shape[1])

    # Assuming df and df_indicator_signals are already defined
    nedded_copies = len(df_indicator_signals.columns)

    # Repeat the 'close' series horizontally to match the shape of df_indicator_signals
    df_dublicate_close = pd.DataFrame(repeat_series_horizontally(df_original['close'], df_indicator_signals.shape), index=df_original.index)

    # Rename columns if shapes match
    if df_indicator_signals.shape == df_dublicate_close.shape:
        df_dublicate_close.columns = df_indicator_signals.columns
        print("Columns have been renamed")
    else:
        print("The DataFrames do not have the same shape.")
    
    
    return df_dublicate_close

# how to access the columns ----> e.g. df_dublicate_close[(0.5, 100, 20, 5, 3,'BTC-USDT')]
df_dublicate_close_pairs ={}
for symbol in symbols:
    df = dataframes[symbol]
    df_dublicate_close = copy_rename_close_series(df, indicator_obj[symbol].limit_order_price)
    df_dublicate_close_pairs[symbol] = df_dublicate_close
    # df_dublicate_close_pairs.append(df_dublicate_close)

print(df_dublicate_close_pairs.keys())
print(df_dublicate_close_pairs[symbols[0]].shape)
print(df_dublicate_close_pairs[symbols[1]].shape)





Columns have been renamed
Columns have been renamed
dict_keys(['ETH-USDT', 'BTC-USDT'])
(554, 12)
(554, 12)


In [125]:
import numpy as np
import vectorbt as vbt
from numba import njit
import talib
from vectorbt.portfolio.enums import SizeType, Direction

# Create arrays to store the data we want to plot
@njit
def order_func_nb(c, high, low, open_, entries, ma_short, entry_price):  # Added entry_price parameter
    close_price = c.close[c.i, c.col]
    close_minus_1bar = c.close[c.i-1, c.col]
    
    # if in position 
    if c.position_now > 0:
        if (close_minus_1bar <= ma_short[c.i-1,c.col]) or (close_price > entry_price[c.i]):
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,
                price=open_[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)
            # Store exit data
            return value

    # if not in position search for position to enter
    elif (c.position_now == 0) and (c.i != 0):
        if (entries[c.i-1,c.col] > 0) and (low[c.i] < entries[c.i-1,c.col]):
            entry_price[:] = np.nan  # Reset entry price array
            entry_price[:] = entries[c.i-1,c.col]  # Update entry price array

            order = vbt.portfolio.nb.order_nb(
                size=1,
                price=entry_price[c.i],
                size_type=SizeType.Percent,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002,
                allow_partial=False,
                raise_reject=True
            )
            return order

    return vbt.portfolio.enums.NoOrder



In [126]:
pf_pairs = {}

for symbol in symbols:
    # candle data
    close = df_dublicate_close_pairs[symbol]
    open_ =dataframes[symbol]['open'].to_numpy().flatten()
    high = dataframes[symbol]['high'].to_numpy().flatten()
    low = dataframes[symbol]['low'].to_numpy().flatten()
    # high = df['high'].to_numpy().flatten()
    # low = df['low'].to_numpy().flatten()

    # indicator data
    entries = indicator_obj[symbol].limit_order_price.to_numpy()
    # entries = df_indicator_signals.to_numpy()
    ma_short = indicator_obj[symbol].short_term_ma.to_numpy()
    # ma_short = indicator.short_term_ma.to_numpy().flatten()

    # Create arrays to store data
    entry_price = np.full(close.shape[0], np.nan)  # Initialize entry price array

    # Create portfolio with trade_data
    pf = vbt.Portfolio.from_order_func(
        close,
        order_func_nb,
        high,
        low,
        open_,
        entries,
        ma_short,
        entry_price,  # Pass entry_price array to the function
        init_cash=500
    )

    pf_pairs[symbol] = pf



In [135]:
# Combine the total return and max drawdown for each symbol
first_glance = []

for symbol in symbols:
    total_return = pf_pairs[symbol].total_return()
    max_dd = pf_pairs[symbol].max_drawdown()
    sharpe_ratio = pf_pairs[symbol].sharpe_ratio()
    sortino_ratio = pf_pairs[symbol].sortino_ratio()
    beta = pf_pairs[symbol].beta()
    profit_factor = pf_pairs[symbol].trades.profit_factor()
    win_rate = pf_pairs[symbol].trades.win_rate()
    total_trades = pf_pairs[symbol].trades.count()
    

    return_and_maxdd = pd.concat([total_return, max_dd,total_trades,sharpe_ratio,sortino_ratio,win_rate,beta,profit_factor], axis=1)


    first_glance.append(return_and_maxdd)

# Concatenate all DataFrames along the rows
combined_df = pd.concat(first_glance, axis=0)

combined_df

total_return  \
custom_strategy_stretch custom_strategy_long_term_ma_len custom_strategy_short_term_ma_len custom_strategy_adx_len custom_strategy_atr_len custom_strategy_symbol                 
0.5                     100                              20                                5                       3                       ETH-USDT                   -0.110209   
                                                                                           6                       3                       ETH-USDT                   -0.110209   
                                                         30                                5                       3                       ETH-USDT                   -0.110209   
                                                                                           6                       3                       ETH-USDT                   -0.110209   
                                                         40                                5                       3                       ETH-USDT                   -0.110209   
                                                                                           6                       3                       ETH-USDT                   -0.110209   
                        200                              20                                5                       3                       ETH-USDT                   -0.097312   
                                                                                           6                       3                       ETH-USDT                   -0.097312   
                                                         30                                5                       3                       ETH-USDT                   -0.097312   
                                                                                           6                       3                       ETH-USDT                   -0.097312   
                                                         40                                5                       3                       ETH-USDT                   -0.097312   
                                                                                           6                       3                       ETH-USDT                   -0.097312   
                        100                              20                                5                       3                       BTC-USDT                   -0.056205   
                                                                                           6                       3                       BTC-USDT                   -0.050075   
                                                         30                                5                       3                       BTC-USDT                   -0.056205   
                                                                                           6                       3                       BTC-USDT                   -0.050075   
                                                         40                                5                       3                       BTC-USDT                   -0.057047   
                                                                                           6                       3                       BTC-USDT                   -0.050922   
                        200                              20                                5                       3                       BTC-USDT                   -0.048385   
                                                                                           6                       3                       BTC-USDT                   -0.042204   
                                                         30                                5                       3                       BTC-USDT                   -0.048385   
                                              

In [150]:
attributes = dir(pf_pairs['ETH-USDT'])
print(attributes)


['__annotations__', '__cached_asset_flow', '__cached_asset_value', '__cached_assets', '__cached_benchmark_returns', '__cached_benchmark_value', '__cached_beta', '__cached_cash', '__cached_cash_flow', '__cached_entry_trades', '__cached_get_entry_trades', '__cached_get_exit_trades', '__cached_get_filled_close', '__cached_get_init_cash', '__cached_get_logs', '__cached_get_orders', '__cached_get_returns_acc', '__cached_get_trades', '__cached_logs', '__cached_max_drawdown', '__cached_orders', '__cached_returns', '__cached_sharpe_ratio', '__cached_sortino_ratio', '__cached_total_profit', '__cached_total_return', '__cached_trades', '__cached_value', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__we

AttributeError: 'Portfolio' object has no attribute 'get_attr'

In [131]:
total_return = pf.total_return()
max_dd = pf.max_drawdown()
total_return
return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
return_and_maxdd.sort_values(by='total_return', ascending=False).head(50)

total_return  \
custom_strategy_stretch custom_strategy_long_term_ma_len custom_strategy_short_term_ma_len custom_strategy_adx_len custom_strategy_atr_len custom_strategy_symbol                 
0.5                     200                              20                                6                       3                       BTC-USDT                   -0.042204   
                                                         30                                6                       3                       BTC-USDT                   -0.042204   
                                                         40                                6                       3                       BTC-USDT                   -0.043058   
                                                         20                                5                       3                       BTC-USDT                   -0.048385   
                                                         30                                5                       3                       BTC-USDT                   -0.048385   
                                                         40                                5                       3                       BTC-USDT                   -0.049234   
                        100                              20                                6                       3                       BTC-USDT                   -0.050075   
                                                         30                                6                       3                       BTC-USDT                   -0.050075   
                                                         40                                6                       3                       BTC-USDT                   -0.050922   
                                                         20                                5                       3                       BTC-USDT                   -0.056205   
                                                         30                                5                       3                       BTC-USDT                   -0.056205   
                                                         40                                5                       3                       BTC-USDT                   -0.057047   

                                                                                                                                                                   max_drawdown  
custom_strategy_stretch custom_strategy_long_term_ma_len custom_strategy_short_term_ma_len custom_strategy_adx_len custom_strategy_atr_len custom_strategy_symbol                
0.5                     200                              20                                6                       3                       BTC-USDT                   -0.052219  
                                                         30                                6                       3                       BTC-USDT                   -0.052219  
                                                         40                                6                       3                       BTC-USDT                   -0.053064  
                                                         20                                5                       3                       BTC-USDT                   -0.063157  
                                                         30                                5                       3                       BTC-USDT                   -0.063157  
                                                         40                                5                       3                       BTC-USDT                   -0.063992  
                        100                              20                                6                       3                       BTC-USDT                   -0.050075  
                                                      